In [1]:
!pip install transformers -q
!pip install tensorflow_datasets


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
  Using cached tensorflow_datasets-4.9.2-py3-none-any.whl (5.4 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 13.9 MB/s eta 0:00:000:00:010:00:01
  Using cached click-8.1.6-py3-none-any.whl (97 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 12.5 MB/s eta 0:00:00
  Using cached etils-1.4.1-py3-none-any.whl (135 kB)
  Using cached promise-2.3.tar.gz (19 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached protobuf-4.23.4-cp37-abi3-macosx_10_9_universal2.whl (400 kB)
  Using cached tensorflow_metadata-1.13.1-py3-none-any.whl (28 kB)
  Using cached termcolor-2.3.0-py3-none-any.whl (6.9 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached wrapt-1.15.0-cp311-cp311-macosx_11_0_

In [6]:
!pip install tensorflow

  Using cached tensorflow-2.13.0-cp311-cp311-macosx_12_0_arm64.whl (1.9 kB)
  Using cached tensorflow_macos-2.13.0-cp311-cp311-macosx_12_0_arm64.whl (189.3 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached h5py-3.9.0-cp311-cp311-macosx_11_0_arm64.whl (2.6 MB)
  Using cached libclang-16.0.6-py2.py3-none-macosx_11_0_arm64.whl (20.6 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached grpcio-1.56.2-cp311-cp311-macosx_10_10_universal2.whl (8.9 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached wheel-0.41.0-py3-none-any.whl (64 kB)
  Using cached googl

In [7]:
import tensorflow_datasets as tfds

In [8]:
dataset, info = tfds.load("oxford_iiit_pet:3.*.*",with_info=True)

In [13]:
dataset

{'train': <_PrefetchDataset element_spec={'file_name': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'segmentation_mask': TensorSpec(shape=(None, None, 1), dtype=tf.uint8, name=None), 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}>,
 'test': <_PrefetchDataset element_spec={'file_name': TensorSpec(shape=(), dtype=tf.string, name=None), 'image': TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None), 'segmentation_mask': TensorSpec(shape=(None, None, 1), dtype=tf.uint8, name=None), 'species': TensorSpec(shape=(), dtype=tf.int64, name=None)}>}

In [14]:
info

tfds.core.DatasetInfo(
    name='oxford_iiit_pet',
    full_name='oxford_iiit_pet/3.2.0',
    description="""
    The Oxford-IIIT pet dataset is a 37 category pet image dataset with roughly 200
    images for each class. The images have large variations in scale, pose and
    lighting. All images have an associated ground truth annotation of breed.
    """,
    homepage='http://www.robots.ox.ac.uk/~vgg/data/pets/',
    data_path='/Users/gmeneses/tensorflow_datasets/oxford_iiit_pet/3.2.0',
    file_format=tfrecord,
    download_size=773.52 MiB,
    dataset_size=774.69 MiB,
    features=FeaturesDict({
        'file_name': Text(shape=(), dtype=string),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=37),
        'segmentation_mask': Image(shape=(None, None, 1), dtype=uint8),
        'species': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=Fal

In [15]:
import tensorflow as tf
from tensorflow.keras import backend

In [16]:
image_size = 512
mean = tf.constant([0.485, 0.456, 0.406])
std = tf.constant([0.229, 0.224, 0.225])


def normalize(input_image, input_mask):
    input_image = tf.image.convert_image_dtype(input_image, tf.float32)
    input_image = (input_image - mean) / tf.maximum(std, backend.epsilon())
    input_mask -= 1
    return input_image, input_mask


def load_image(datapoint):
    input_image = tf.image.resize(datapoint["image"], (image_size, image_size))
    input_mask = tf.image.resize(
        datapoint["segmentation_mask"],
        (image_size, image_size),
        method="bilinear",
    )

    input_image, input_mask = normalize(input_image, input_mask)
    input_image = tf.transpose(input_image, (2, 0, 1))
    return {"pixel_values": input_image, "labels": tf.squeeze(input_mask)}

In [19]:
# checking one image ans mask of the tensowflow dataset
sample = next(iter(dataset["train"]))
image, mask = sample["image"], sample["segmentation_mask"]

# Inspect the shape and data type of the image and mask
print("Image shape:", image.shape)  # E.g., (height, width, channels)
print("Image data type:", image.dtype)  # E.g., float32

print("Mask shape:", mask.shape)  # E.g., (height, width, 1)
print("Mask data type:", mask.dtype)  # E.g., int64

Image shape: (500, 500, 3)
Image data type: <dtype: 'uint8'>
Mask shape: (500, 500, 1)
Mask data type: <dtype: 'uint8'>


In [46]:
def convert_mask_to_single_channel(mask_3_channels):
    # Assuming mask_3_channels has shape (height, width, 3)
    height, width, _ = mask_3_channels.shape

    # Create an empty array with shape (height, width, 1) for the single-channel mask
    single_channel_mask = np.zeros((height, width, 1), dtype=np.uint8)

    # Define the colors representing each category (RGB values)
    category_colors = {
        (0, 0, 0): 0,        # Class 0 - Black (no building) or un-classified
        (255, 255, 255): 1,  # Class 1 - White (no-damage)
        (255, 0, 0): 2,      # Class 2 - Red (minor damage)
        (0, 255, 0): 3,      # Class 3 - Green (major damage)
        (0, 0, 255): 4,      # Class 4 - Blue (destroyed)
    }
    # Loop through each pixel and assign the corresponding category to the single-channel mask
    for y in range(height):
        for x in range(width):
            pixel_color = tuple(mask_3_channels[y, x])
            category = category_colors.get(pixel_color, -1)  # -1 for unknown category
            single_channel_mask[y, x] = category

    return single_channel_mask

In [47]:
from PIL import Image
import numpy as np

def read_png_image(file_path):
    image = Image.open(file_path)
    image_array = np.array(image)
    return image_array

In [48]:
# need to change this it's too slow. Here I'm just working with a subset of guatemala-volcano
from os import path, walk, makedirs
from tqdm import tqdm

path_example="/Users/gmeneses/DScourse/00_capstone/xView2_baseline_fork/xBD_last_subset_test_mask"
disasters = next(walk(path_example))[1]

image_arrays = []
mask_arrays = []
for disaster in tqdm(disasters, desc='Creating image and mask arrays', unit='disaster'):
    # Create the full path to the images, labels, and mask output directories
    image_dir = path.join(path_example, disaster, 'images')
    mask_dir = path.join(path_example, disaster, 'masks')

    if not path.isdir(image_dir):
        print(
            "Error, could not find image files in {}.\n\n"
            .format(image_dir),
            file=stderr)
        exit(2)

    if not path.isdir(mask_dir):
        print(
            "Error, could not find labels in {}.\n\n"
            .format(json_dir),
            file=stderr)
        exit(3)
    
        
    # running through masks because it can be that there are no masks for certain images ()
    # in this case masks have the same name than images
    masks_list = [j for j in next(walk(mask_dir))[2] if '_post' in j]
    for im in masks_list:
        img_pre_name = path.splitext(im.replace('_post', '_pre'))[0] + '.png'
        img_post_name = im
        mask_name = im
        # path to images and mask
        img_pre = path.join(image_dir,img_pre_name)
        img_post = path.join(image_dir,img_post_name)
        mask = path.join(mask_dir,mask_name)
        
        array_pre = read_png_image(img_pre)
        array_post = read_png_image(img_post)
        array_mask_3d = read_png_image(mask)
        
        # converting mask to depth 1
        array_mask = convert_mask_to_single_channel(array_mask_3d)
        # creating a final image array (1024x1024x6) 
        array_image = np.concatenate((array_pre, array_post), axis=-1)
        # adding to lists in array format
        image_arrays.append(array_image)
        mask_arrays.append(array_mask)

        
        
        


    
    

Creating image and mask arrays: 100%|████████████████████████████████████████████████████| 1/1 [00:34<00:00, 34.54s/disaster]


In [49]:
# Convert lists to numpy arrays
images = np.stack(image_arrays)
masks = np.stack(mask_arrays)

print(images.shape)
print(masks.shape)

(12, 1024, 1024, 6)
(12, 1024, 1024, 1)


In [50]:
# saving the arrays in npz numpy format 
np.savez('input_model.npz', images=images, masks=masks)

In [51]:
#to recover images and mask arrays:
loaded_arrays = np.load('input_model.npz')
images = loaded_arrays['images']
masks = loaded_arrays['masks']

In [62]:
# to see the pixel values of only mask number 5
print(masks[5, :, :, 0])


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [63]:
# see the values that fill the mask number 5 
print(np.unique(masks[5, :, :, 0])) # [0 1 4] --> only buildings with damage levels 1 and 4 in this

[0 1 4]
